In [1]:
!pip install kaggle --quiet

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/keys/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split, Dataset
from PIL import Image
import time
# Configurar dispositivo (GPU si está disponible)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

Usando dispositivo: cuda:0


In [5]:
# --- Celda 2: Subir tu Framework de Ataque ---
!mkdir -p attacks
!mkdir -p models

print("\nPor favor, sube tus archivos .py del framework usando el panel de la izquierda.")
print("Sube a 'attacks/': base.py, carlini_wagner.py")
print("Sube a 'models/': base_models.py, pytorch.py")
print("Sube al directorio raíz: adversarial.py, criteria.py, distances.py, utils.py")


Por favor, sube tus archivos .py del framework usando el panel de la izquierda.
Sube a 'attacks/': base.py, carlini_wagner.py
Sube a 'models/': base_models.py, pytorch.py
Sube al directorio raíz: adversarial.py, criteria.py, distances.py, utils.py


In [6]:
import os
from google.colab import files

In [7]:
# --- Celda 3: Descarga y Preparación del Dataset (CORREGIDA) ---
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split, Dataset
from PIL import Image
import os

# Descargar y descomprimir el dataset
!kaggle competitions download -c sci-pediatric-chest-x-ray-pneumonia-classificatio --force -q
!unzip -oq "sci-pediatric-chest-x-ray-pneumonia-classificatio.zip"

# --- RUTAS CORRECTAS ---
train_dir = 'train/train'  # Apunta al directorio que contiene las carpetas NORMAL/PNEUMONIA
test_dir = 'test/test'      # Apunta al directorio con todas las imágenes de prueba juntas

print(f"Directorio de entrenamiento usado: {train_dir}")
print(f"Directorio de prueba usado: {test_dir}")

# --- Transformaciones de Imagen para ResNet ---
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Grayscale(num_output_channels=3), # ResNet espera 3 canales
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val_test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# --- LÓGICA DE SPLIT (imitando a TensorFlow) ---
full_train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms['val_test'])
train_size = int(0.8 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])
train_dataset.dataset.transform = data_transforms['train'] # Aplicar aumentación solo a train

# --- CLASE DATASET PERSONALIZADA PARA EL TEST SET (sin carpetas de clase) ---
class UnlabeledImageFolder(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = sorted([f for f in os.listdir(root_dir) if os.path.isfile(os.path.join(root_dir, f))])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.image_files[idx])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        # Devolvemos un label ficticio (ej. -1) porque el dataloader lo requiere, pero no lo usaremos
        return image, -1

test_dataset = UnlabeledImageFolder(test_dir, transform=data_transforms['val_test'])

# --- CREAR DATALOADERS ---
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2),
    'val': DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2),
    'test': DataLoader(test_dataset, batch_size=1, shuffle=False) # No barajar para mantener el orden
}

class_names = full_train_dataset.classes
print(f"\nClases encontradas: {class_names}")
print(f"Tamaño del set de entrenamiento: {len(train_dataset)}")
print(f"Tamaño del set de validación: {len(val_dataset)}")
print(f"Tamaño del set de prueba: {len(test_dataset)}")

Directorio de entrenamiento usado: train/train
Directorio de prueba usado: test/test

Clases encontradas: ['NORMAL', 'PNEUMONIA']
Tamaño del set de entrenamiento: 3052
Tamaño del set de validación: 764
Tamaño del set de prueba: 2040


In [8]:
# --- Transformaciones de Imagen para ResNet ---
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Grayscale(num_output_channels=3), # ResNet espera 3 canales
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val_test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# --- LÓGICA DE SPLIT ---
full_train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms['val_test'])
train_size = int(0.8 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])
train_dataset.dataset.transform = data_transforms['train']

# --- CLASE DATASET PERSONALIZADA PARA EL TEST SET ---
class UnlabeledImageFolder(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = sorted([f for f in os.listdir(root_dir) if os.path.isfile(os.path.join(root_dir, f))])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.image_files[idx])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        # Devolvemos un label ficticio (ej. -1), no lo usaremos
        return image, -1

test_dataset = UnlabeledImageFolder(test_dir, transform=data_transforms['val_test'])

# --- CREAR DATALOADERS Y DICCIONARIO DE DATASETS ---
image_datasets = {'train': train_dataset, 'val': val_dataset, 'test': test_dataset}
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2),
    'val': DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2),
    'test': DataLoader(test_dataset, batch_size=1, shuffle=False)
}

class_names = full_train_dataset.classes
print(f"\nClases encontradas: {class_names}")
print(f"Tamaño del set de entrenamiento: {len(train_dataset)}")
print(f"Tamaño del set de validación: {len(val_dataset)}")
print(f"Tamaño del set de prueba: {len(test_dataset)}")


Clases encontradas: ['NORMAL', 'PNEUMONIA']
Tamaño del set de entrenamiento: 3052
Tamaño del set de validación: 764
Tamaño del set de prueba: 2040


In [12]:
import torch
from torchvision import models
from torchvision.models import inception_v3, Inception_V3_Weights

# 1) Selección de dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2) Instanciar y cargar modelos preentrenados
resnet50_model        = models.resnet50(pretrained=True).to(device).eval()
densenet121_model     = models.densenet121(pretrained=True).to(device).eval()
efficientnet_b0_model = models.efficientnet_b0(pretrained=True).to(device).eval()
inception_v3_model    = inception_v3(weights=Inception_V3_Weights.IMAGENET1K_V1).to(device).eval()
vgg19_model           = models.vgg19(pretrained=True).to(device).eval()

# 3) Diccionario de modelos a evaluar
models = {
    "ResNet50":        resnet50_model,
    "DenseNet121":     densenet121_model,
    "EfficientNetB0":  efficientnet_b0_model,
    "InceptionV3":     inception_v3_model,
    "VGG19":           vgg19_model,
}

Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:01<00:00, 86.9MB/s] 
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:06<00:00, 83.3MB/s]


In [15]:
# Mean/std de ImageNet para canales RGB
imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_std  = np.array([0.229, 0.224, 0.225])

# Cálculo de bounds: min y max tras normalizar [0,1]
min_val = float(((0.0 - imagenet_mean) / imagenet_std).min())
max_val = float(((1.0 - imagenet_mean) / imagenet_std).max())
bounds  = (min_val, max_val)

# Número de clases de tu detector de tomografías
NUM_CLASSES = 2   # ajusta si tienes más clases

# Ahora sí puedes instanciar PyTorchModel sin NameError:
fmodel = PyTorchModel(
    model.to(device).eval(),
    bounds=bounds,
    num_classes=NUM_CLASSES,
    channel_axis=1,
    device=device
)


In [17]:
# --- Celda 4: Entrenamiento de Múltiples Modelos ---
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import time, copy

# 1) Configurar dispositivo y parámetros generales
device     = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_epochs = 10
lr         = 1e-3
criterion  = nn.CrossEntropyLoss()

# 2) Definir los modelos a entrenar (pre-trained) y su nombre de archivo de guardado
model_constructors = {
    "resnet18":   lambda: models.resnet18(pretrained=True),
    "resnet50":   lambda: models.resnet50(pretrained=True),
    "densenet121":lambda: models.densenet121(pretrained=True),
    "efficientnet_b0": lambda: models.efficientnet_b0(pretrained=True),
    "vgg19":      lambda: models.vgg19(pretrained=True),
}

for name, constructor in model_constructors.items():
    print(f"\n=== Entrenando {name} ===")

    # 3) Instanciar y adaptar la última capa
    model = constructor()
    if "resnet" in name:
        num_ftrs       = model.fc.in_features
        model.fc       = nn.Linear(num_ftrs, len(class_names))
    elif "densenet" in name:
        num_ftrs       = model.classifier.in_features
        model.classifier = nn.Linear(num_ftrs, len(class_names))
    elif "efficientnet" in name:
        num_ftrs        = model.classifier[1].in_features
        model.classifier[1] = nn.Linear(num_ftrs, len(class_names))
    elif "vgg" in name:
        num_ftrs       = model.classifier[-1].in_features
        model.classifier[-1] = nn.Linear(num_ftrs, len(class_names))
    else:  # inception_v3
        num_ftrs       = model.fc.in_features
        model.fc       = nn.Linear(num_ftrs, len(class_names))
        # si usas aux_logits=True, también ajustarías model.AuxLogits.fc

    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # 4) Bucle de entrenamiento / validación
    best_acc       = 0.0
    save_path      = f"{name}_best.pth"
    since          = time.time()

    for epoch in range(num_epochs):
        print(f"[{name}] Epoch {epoch+1}/{num_epochs}")
        for phase in ("train", "val"):
            model.train() if phase=="train" else model.eval()
            running_loss, running_corrects = 0.0, 0

            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase=="train"):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    if phase=="train":
                        loss.backward()
                        optimizer.step()
                running_loss     += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc  = running_corrects.double() / len(image_datasets[phase])
            print(f" {phase}  Loss: {epoch_loss:.4f}  Acc: {epoch_acc:.4f}")

            # Guardar mejor modelo de validación
            if phase=="val" and epoch_acc > best_acc:
                best_acc = epoch_acc
                torch.save(model.state_dict(), save_path)
                print(f"  -> Nuevo mejor modelo guardado: {save_path}")

    elapsed = time.time() - since
    print(f"[{name}] Entrenamiento completado en {elapsed//60:.0f}m {elapsed%60:.0f}s, Mejor Acc: {best_acc:.4f}")


=== Entrenando resnet18 ===


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 83.6MB/s]


[resnet18] Epoch 1/10
 train  Loss: 0.1891  Acc: 0.9292
 val  Loss: 0.1336  Acc: 0.9411
  -> Nuevo mejor modelo guardado: resnet18_best.pth
[resnet18] Epoch 2/10
 train  Loss: 0.1363  Acc: 0.9499
 val  Loss: 0.1177  Acc: 0.9542
  -> Nuevo mejor modelo guardado: resnet18_best.pth
[resnet18] Epoch 3/10
 train  Loss: 0.1031  Acc: 0.9620
 val  Loss: 0.4192  Acc: 0.8809
[resnet18] Epoch 4/10
 train  Loss: 0.0903  Acc: 0.9685
 val  Loss: 0.1069  Acc: 0.9620
  -> Nuevo mejor modelo guardado: resnet18_best.pth
[resnet18] Epoch 5/10
 train  Loss: 0.0936  Acc: 0.9682
 val  Loss: 0.1070  Acc: 0.9555
[resnet18] Epoch 6/10
 train  Loss: 0.0722  Acc: 0.9744
 val  Loss: 0.1159  Acc: 0.9634
  -> Nuevo mejor modelo guardado: resnet18_best.pth
[resnet18] Epoch 7/10
 train  Loss: 0.0989  Acc: 0.9636
 val  Loss: 0.0988  Acc: 0.9607
[resnet18] Epoch 8/10
 train  Loss: 0.0777  Acc: 0.9715
 val  Loss: 0.1291  Acc: 0.9463
[resnet18] Epoch 9/10
 train  Loss: 0.0694  Acc: 0.9767
 val  Loss: 0.0797  Acc: 0.9699


In [20]:
# --- Celda 5: Generación de Ejemplos Adversariales para Todos los Modelos Entrenados ---
import os, sys
import numpy as np
import torch
from torchvision import models
from tqdm.notebook import tqdm

# Asegúrate de añadir tu carpeta de trabajo al path para encontrar tus módulos
sys.path.insert(0, os.getcwd())

from models.pytorch import PyTorchModel
from attacks.carlini_wagner import CarliniWagnerL2Attack
from criteria import Misclassification
from distances import MeanSquaredDistance as MSE
from adversarial import Adversarial

# 1) Configuración general
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
N_SAMPLES = 100  # Número de adversariales por modelo
hypers = dict(binary_search_steps=5, max_iterations=100, learning_rate=1e-2)

# 2) Definir modelos y rutas a sus pesos
#    Cada constructor debe luego ajustar la capa final antes de cargar pesos
model_defs = {
    "resnet18":   (models.resnet18,   "resnet18_best.pth"),
    "resnet50":   (models.resnet50,   "resnet50_best.pth"),
    "densenet121":(models.densenet121,"densenet121_best.pth"),
    "efficientnet_b0": (models.efficientnet_b0, "efficientnet_b0_best.pth"),
    "vgg19":      (models.vgg19,      "vgg19_best.pth"),
}

# 3) Mean/std y bounds de ImageNet
mean = np.array([0.485, 0.456, 0.406])
std  = np.array([0.229, 0.224, 0.225])
min_val, max_val = float(((0 - mean)/std).min()), float(((1 - mean)/std).max())
bounds = (min_val, max_val)

# 4) Recoger resultados
results = {}

for name, (constructor, weight_path) in model_defs.items():
    print(f"\n--- Generando adv para {name} ---")
    # Instanciar modelo pre-trained=False y ajustar fc/classifier
    model = constructor(pretrained=False)
    num_classes = len(class_names)
    if name.startswith("resnet"):
        in_f = model.fc.in_features
        model.fc = torch.nn.Linear(in_f, num_classes)
    elif name.startswith("densenet"):
        in_f = model.classifier.in_features
        model.classifier = torch.nn.Linear(in_f, num_classes)
    elif name.startswith("efficientnet"):
        in_f = model.classifier[1].in_features
        model.classifier[1] = torch.nn.Linear(in_f, num_classes)
    elif name.startswith("vgg"):
        in_f = model.classifier[-1].in_features
        model.classifier[-1] = torch.nn.Linear(in_f, num_classes)
    # Mover e inspeccionar
    model.load_state_dict(torch.load(weight_path, map_location=device))
    model.to(device).eval()

    # Wrappear para adversarial framework
    fmodel = PyTorchModel(model,
                          bounds=bounds,
                          num_classes=num_classes,
                          channel_axis=1,
                          device=device)

    # Preparar el ataque
    attack = CarliniWagnerL2Attack()

    # Listas temporales para cada modelo
    benign_images, adv_images, distances = [], [], []
    count = 0

    # Generación con tqdm
    for data, _ in tqdm(dataloaders['test'], desc=f"{name} adv gen"):
        if count >= N_SAMPLES:
            break
        # De tensor a numpy y etiqueta original
        img_np = data.squeeze(0).permute(1,2,0).cpu().numpy()
        with torch.no_grad():
            orig_lbl = int(model(data.to(device)).argmax(dim=1))

        # Crear y ejecutar ataque
        adv = Adversarial(fmodel, Misclassification(), img_np, orig_lbl, distance=MSE)
        attack(adv, **hypers)

        # Si exitoso, almacenar
        if adv.image is not None:
            benign_images.append(img_np)
            adv_images.append(adv.image)
            distances.append(adv.distance.value)
            count += 1

    results[name] = dict(
        benign=benign_images,
        adversarial=adv_images,
        distance=distances
    )
    print(f"{name}: generados {count}/{N_SAMPLES}")




--- Generando adv para resnet18 ---


resnet18 adv gen:   0%|          | 0/2040 [00:00<?, ?it/s]


Búsqueda binaria paso 1/5, const = 1.0e-03
  Iteración 100/100, Pérdida: 0.0037, Mejor L2: 2.5413
  Éxito para const=1.0e-03. Nuevo límite superior: 1.0e-03

Búsqueda binaria paso 2/5, const = 5.0e-04
  Iteración 100/100, Pérdida: 0.0026, Mejor L2: 2.6460
  Éxito para const=5.0e-04. Nuevo límite superior: 5.0e-04

Búsqueda binaria paso 3/5, const = 2.5e-04
  Iteración 100/100, Pérdida: 0.0020, Mejor L2: 3.7092
  Éxito para const=2.5e-04. Nuevo límite superior: 2.5e-04

Búsqueda binaria paso 4/5, const = 1.3e-04
  Iteración 100/100, Pérdida: 0.0018, Mejor L2: 7.2026
  Éxito para const=1.3e-04. Nuevo límite superior: 1.3e-04

Búsqueda binaria paso 5/5, const = 6.3e-05
  Iteración 100/100, Pérdida: 0.0016, Mejor L2: inf
  Fallo para const=6.3e-05. Nuevo límite inferior: 6.3e-05

Búsqueda binaria paso 1/5, const = 1.0e-03
  Iteración 100/100, Pérdida: 0.0143, Mejor L2: inf
  Fallo para const=1.0e-03. Nuevo límite inferior: 1.0e-03

Búsqueda binaria paso 2/5, const = 1.0e-02
  Iteración 10

/content/attacks/base.py:145: UserWarning: CarliniWagnerL2Attack did not find an adversarial, maybe the model or the criterion is not supported by this attack.
  warnings.warn('{} did not find an adversarial, maybe the model'


  Iteración 100/100, Pérdida: 0.0090, Mejor L2: 7.4900
  Éxito para const=1.0e-03. Nuevo límite superior: 1.0e-03

Búsqueda binaria paso 2/5, const = 5.0e-04
  Iteración 100/100, Pérdida: 0.0053, Mejor L2: inf
  Fallo para const=5.0e-04. Nuevo límite inferior: 5.0e-04

Búsqueda binaria paso 3/5, const = 7.5e-04
  Iteración 100/100, Pérdida: 0.0071, Mejor L2: inf
  Fallo para const=7.5e-04. Nuevo límite inferior: 7.5e-04

Búsqueda binaria paso 4/5, const = 8.8e-04
  Iteración 100/100, Pérdida: 0.0081, Mejor L2: 7.4866
  Éxito para const=8.8e-04. Nuevo límite superior: 8.8e-04

Búsqueda binaria paso 5/5, const = 8.1e-04
  Iteración 100/100, Pérdida: 0.0076, Mejor L2: inf
  Fallo para const=8.1e-04. Nuevo límite inferior: 8.1e-04

Búsqueda binaria paso 1/5, const = 1.0e-03
  Iteración 100/100, Pérdida: 0.0058, Mejor L2: 7.8416
  Éxito para const=1.0e-03. Nuevo límite superior: 1.0e-03

Búsqueda binaria paso 2/5, const = 5.0e-04
  Iteración 100/100, Pérdida: 0.0038, Mejor L2: 7.8201
  Éxit

resnet50 adv gen:   0%|          | 0/2040 [00:00<?, ?it/s]


Búsqueda binaria paso 1/5, const = 1.0e-03
  Iteración 100/100, Pérdida: 0.0041, Mejor L2: 1.8352
  Éxito para const=1.0e-03. Nuevo límite superior: 1.0e-03

Búsqueda binaria paso 2/5, const = 5.0e-04
  Iteración 100/100, Pérdida: 0.0028, Mejor L2: 1.9096
  Éxito para const=5.0e-04. Nuevo límite superior: 5.0e-04

Búsqueda binaria paso 3/5, const = 2.5e-04
  Iteración 100/100, Pérdida: 0.0021, Mejor L2: 2.5695
  Éxito para const=2.5e-04. Nuevo límite superior: 2.5e-04

Búsqueda binaria paso 4/5, const = 1.3e-04
  Iteración 100/100, Pérdida: 0.0018, Mejor L2: 3.7023
  Éxito para const=1.3e-04. Nuevo límite superior: 1.3e-04

Búsqueda binaria paso 5/5, const = 6.3e-05
  Iteración 100/100, Pérdida: 0.0017, Mejor L2: 7.1958
  Éxito para const=6.3e-05. Nuevo límite superior: 6.3e-05

Búsqueda binaria paso 1/5, const = 1.0e-03
  Iteración 100/100, Pérdida: 0.0108, Mejor L2: inf
  Fallo para const=1.0e-03. Nuevo límite inferior: 1.0e-03

Búsqueda binaria paso 2/5, const = 1.0e-02
  Iteración

densenet121 adv gen:   0%|          | 0/2040 [00:00<?, ?it/s]


Búsqueda binaria paso 1/5, const = 1.0e-03
  Iteración 100/100, Pérdida: 0.0031, Mejor L2: 0.8940
  Éxito para const=1.0e-03. Nuevo límite superior: 1.0e-03

Búsqueda binaria paso 2/5, const = 5.0e-04
  Iteración 100/100, Pérdida: 0.0023, Mejor L2: 1.0142
  Éxito para const=5.0e-04. Nuevo límite superior: 5.0e-04

Búsqueda binaria paso 3/5, const = 2.5e-04
  Iteración 100/100, Pérdida: 0.0019, Mejor L2: 0.9923
  Éxito para const=2.5e-04. Nuevo límite superior: 2.5e-04

Búsqueda binaria paso 4/5, const = 1.3e-04
  Iteración 100/100, Pérdida: 0.0017, Mejor L2: 1.2126
  Éxito para const=1.3e-04. Nuevo límite superior: 1.3e-04

Búsqueda binaria paso 5/5, const = 6.3e-05
  Iteración 100/100, Pérdida: 0.0016, Mejor L2: 1.8055
  Éxito para const=6.3e-05. Nuevo límite superior: 6.3e-05

Búsqueda binaria paso 1/5, const = 1.0e-03
  Iteración 100/100, Pérdida: 0.0072, Mejor L2: 7.7421
  Éxito para const=1.0e-03. Nuevo límite superior: 1.0e-03

Búsqueda binaria paso 2/5, const = 5.0e-04
  Iterac

KeyboardInterrupt: 

In [21]:
# --- Celda 6: Guardar Todos los Resultados y Pesos en Google Drive ---

import os
import shutil
import numpy as np
from google.colab import drive

# 1) Montar Google Drive
if not os.path.ismount('/content/drive'):
    drive.mount('/content/drive')

# 2) Directorio base en Drive
save_dir = '/content/drive/MyDrive/XRay_Adversarial_Data/'
os.makedirs(save_dir, exist_ok=True)

# 3) Guardar arrays de cada modelo
for name, res in results.items():
    benign_arr = np.stack(res['benign'], axis=0) if res['benign'] else np.empty((0,))
    adv_arr    = np.stack(res['adversarial'], axis=0) if res['adversarial'] else np.empty((0,))
    dist_arr   = np.array(res['distance'])

    np.save(os.path.join(save_dir, f"{name}_benign.npy"),        benign_arr)
    np.save(os.path.join(save_dir, f"{name}_adversarial.npy"),   adv_arr)
    np.save(os.path.join(save_dir, f"{name}_distances.npy"),     dist_arr)

# 4) Copiar checkpoints (.pth) de cada modelo
for name, (_, weight_path) in model_defs.items():
    if os.path.exists(weight_path):
        dest_path = os.path.join(save_dir, os.path.basename(weight_path))
        shutil.copy(weight_path, dest_path)
        print(f"Checkpoint guardado: {dest_path}")

# 5) Informar archivos guardados
print("\nArchivos en:", save_dir)
for fname in sorted(os.listdir(save_dir)):
    print(" -", fname)


Checkpoint guardado: /content/drive/MyDrive/XRay_Adversarial_Data/resnet18_best.pth
Checkpoint guardado: /content/drive/MyDrive/XRay_Adversarial_Data/resnet50_best.pth
Checkpoint guardado: /content/drive/MyDrive/XRay_Adversarial_Data/densenet121_best.pth
Checkpoint guardado: /content/drive/MyDrive/XRay_Adversarial_Data/efficientnet_b0_best.pth
Checkpoint guardado: /content/drive/MyDrive/XRay_Adversarial_Data/vgg19_best.pth

Archivos en: /content/drive/MyDrive/XRay_Adversarial_Data/
 - adv_distances_xray.npy
 - adv_images_xray.npy
 - benign_images_xray.npy
 - densenet121_best.pth
 - efficientnet_b0_best.pth
 - resnet18_adversarial.npy
 - resnet18_benign.npy
 - resnet18_best.pth
 - resnet18_distances.npy
 - resnet18_xray_best.pth
 - resnet50_adversarial.npy
 - resnet50_benign.npy
 - resnet50_best.pth
 - resnet50_distances.npy
 - vgg19_best.pth
